In [13]:
import requests
import pandas as pd
import numpy as np
import urllib
import zipfile
pd.set_option('display.max_columns', 500)

import matplotlib.pylab as pl
from pandas.plotting import scatter_matrix
import json
import geopandas as gpd
import fiona
import shapely

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [14]:
url = 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyczoningtaxlotdb_20190331.zip'

In [3]:
urllib.urlretrieve (url, "nyczoningtaxlotdb_20190331.zip")

('nyczoningtaxlotdb_20190331.zip',
 <httplib.HTTPMessage instance at 0x000000000A62AB88>)

In [4]:
zip_ref = zipfile.ZipFile("nyczoningtaxlotdb_20190331.zip", 'r')
zip_ref.extractall("nyczoningtaxlotdb_20190331")
zip_ref.close()

In [5]:
NY_Zoning = pd.read_csv("nyczoningtaxlotdb_20190331\NY_ZoningTaxLotDB20190331.csv")

C:\Users\sherr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (5,6,7,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
NY_Zoning.head()

,Borough Code,Tax Block,Tax Lot,BBL,Zoning District 1,Zoning District 2,Zoning District 3,Zoning District 4,Commercial Overlay 1,Commercial Overlay 2,Special District 1,Special District 2,Special District 3,Limited Height District,Zoning Map Number,Zoning Map Code
0,1.0,1234,19,1012340019,R10A,C4-6A,R8,C1-9,C1-5,NaN,EC-2,EC-3,NaN,NaN,5D,NaN
1,2.0,2833,30,2028330030,R8,R8A,NaN,NaN,C2-4,NaN,C,J,NaN,NaN,3B,NaN
2,1.0,1666,119,1016660119,R7D,R9A,NaN,NaN,C1-5,C2-5,EHC,EHC/TA,NaN,NaN,6B,NaN
3,1.0,1665,130,1016650130,R7D,R9A,NaN,NaN,C1-5,C2-5,EHC,EHC/TA,NaN,NaN,6B,NaN
4,3.0,4141,26,3041410026,M1-4/R7A,R6B,NaN,NaN,C2-4,NaN,EC-6,MX-16,NaN,NaN,17C,NaN


Zoning District repersents the zoning classification occupying the greatest percentage of the tax lot's area.

1 means greatest, 4 means fourth greatest.

(Only zoning districts that cover at least 10% of a tax lot’s area are included.)

Commercial Overlay represents the commercial overlay occupying the greatest percentage of the lot area. 

(The commercial overlay district must either cover at least 10% of a tax lot’s area or at least 50% of the commercial overlay district must be contained within the tax lot.)


Special District 1 represents the special purpose district occupying the greatest percentage of the lot area.

Limited height districts are coded using the three to five character district symbols that are listed in Appendix E.

The Zoning Map Number associated with the tax lot.

Zoning Map Number is one of the potential Zoning Maps associated with the Tax Lot.

In [7]:
reahook_Zoning = NY_Zoning[NY_Zoning['Zoning Map Number'] == '16A']
reahook_Zoning.reset_index(drop = True, inplace = True)

In [8]:
reahook_Zoning.head()

,Borough Code,Tax Block,Tax Lot,BBL,Zoning District 1,Zoning District 2,Zoning District 3,Zoning District 4,Commercial Overlay 1,Commercial Overlay 2,Special District 1,Special District 2,Special District 3,Limited Height District,Zoning Map Number,Zoning Map Code
0,3.0,330,9,3003300009,R6A,NaN,NaN,NaN,C2-4,NaN,NaN,NaN,NaN,NaN,16A,NaN
1,3.0,610,28,3006100028,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
2,3.0,610,3,3006100003,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
3,3.0,597,35,3005970035,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
4,3.0,603,14,3006030014,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN


In [9]:
reahook_Zoning = reahook_Zoning[(reahook_Zoning["Tax Block"] == 554) | (reahook_Zoning["Tax Block"] == 555)
                |(reahook_Zoning["Tax Block"] == 556) |(reahook_Zoning["Tax Block"] == 564)
                |(reahook_Zoning["Tax Block"] == 565) |(reahook_Zoning["Tax Block"] == 566)
                |(reahook_Zoning["Tax Block"] == 574) |(reahook_Zoning["Tax Block"] == 575)
                |(reahook_Zoning["Tax Block"] == 576) |(reahook_Zoning["Tax Block"] == 585)
                |(reahook_Zoning["Tax Block"] == 586) |(reahook_Zoning["Tax Block"] == 587)
                |(reahook_Zoning["Tax Block"] == 596) |(reahook_Zoning["Tax Block"] == 597)
                |(reahook_Zoning["Tax Block"] == 598) |(reahook_Zoning["Tax Block"] == 603)
                |(reahook_Zoning["Tax Block"] == 604)]

In [15]:
reahook_Zoning.head()

,Borough Code,Tax Block,Tax Lot,BBL,Zoning District 1,Zoning District 2,Zoning District 3,Zoning District 4,Commercial Overlay 1,Commercial Overlay 2,Special District 1,Special District 2,Special District 3,Limited Height District,Zoning Map Number,Zoning Map Code
3,3.0,597,35,3005970035,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
4,3.0,603,14,3006030014,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
10,3.0,597,27,3005970027,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
12,3.0,603,23,3006030023,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN
13,3.0,603,8,3006030008,M1-1/R5,NaN,NaN,NaN,NaN,NaN,MX-5,NaN,NaN,NaN,16A,NaN


In [26]:
reahook_Zoning.shape

(381, 16)

In [11]:
redhook = gpd.GeoDataFrame.from_file("redhook/redhook.shp")

In [27]:
redhook.shape

(370, 90)

In [16]:
redhook.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,Shape_Leng,Shape_Area,geometry
0,BK,554,1,306,53,1008,15,38,11231,L101,76,38,4100,3,06,1D,23 FERRIS STREET,M2-1,None,None,None,None,None,None,None,None,None,N,G1,10,0,None,"41 FERRIS, LLC",20000,5820,5820,0,0,0,5820,0,0,0,2,1,1.0,0,1,200.0,100.0,60.0,96.83,None,0,N,3,0,149850.0,929700.0,0.0,0.0,1931,0,0,None,None,0.29,0.0,2.0,0.0,3,3.005540e+09,0,0053,980224,186962,16a,None,301 003,30206,None,0.0,None,1,1,1,18v2.1,605.316103,20280.353102,"POLYGON ((980329.5232489705 187004.2464481443,..."
1,BK,554,12,306,53,1008,15,38,11231,L101,76,38,4100,3,06,1D,209 KING STREET,M2-1,None,None,None,None,None,None,None,None,None,N,G7,10,0,None,MIRIAM (FESTINGER) SO,5000,0,0,0,0,0,0,0,0,0,7,2,0.0,0,0,50.0,100.0,0.0,0.00,None,0,N,5,5,97650.0,97650.0,0.0,0.0,0,0,0,None,None,0.00,0.0,2.0,0.0,3,3.005540e+09,0,0053,980314,186948,16a,None,301 003,30206,None,0.0,None,1,1,1,18v2.1,310.986438,5402.061762,"POLYGON ((980368.7078818828 186969.4561633915,..."
2,BK,554,14,306,53,1008,15,38,11231,L101,76,38,4100,3,06,1D,199 KING STREET,M2-1,None,None,None,None,None,None,None,None,None,N,G1,10,0,None,ECLECTIC PROPERTIES G,25000,24575,24575,0,0,0,24575,0,0,0,2,4,1.0,0,1,150.0,200.0,90.0,200.00,None,0,Y,4,0,393750.0,1021950.0,0.0,0.0,1931,0,0,None,None,0.98,0.0,2.0,0.0,3,3.005540e+09,0,0053,980318,186863,16a,None,301 003,30206,None,0.0,None,1,1,1,18v2.1,708.334568,25251.225071,"POLYGON ((980442.3661991358 186904.057672143, ..."
3,BK,554,27,306,53,1008,15,38,11231,L101,76,38,4100,3,06,1D,142 CONOVER STREET,M2-1,None,None,None,None,None,None,None,None,None,N,E1,06,0,None,COLLINS JENNIFER L,2500,2500,2500,0,0,0,0,2500,0,0,2,1,1.0,0,1,25.0,100.0,25.0,100.00,None,2,N,3,0,20250.0,106200.0,0.0,0.0,1961,0,0,None,None,1.00,0.0,2.0,0.0,3,3.005540e+09,0,0053,980585,186759,16a,None,301 003,30206,None,0.0,None,1,1,1,18v2.1,258.233187,2765.747175,"POLYGON ((980614.0479423851 186715.031475395, ..."
4,BK,554,28,306,53,1008,15,38,11231,L101,76,38,4100,3,06,1D,144 CONOVER STREET,M2-1,None,None,None,None,None,None,None,None,None,N,C1,02,0,None,144 CONOVER STREET LL,2500,5872,0,5872,0,0,0,0,0,0,2,1,4.0,8,8,25.0,100.0,25.0,60.00,None,0,N,5,2,1267.0,89246.0,0.0,0.0,1921,0,0,None,None,2.35,0.0,2.0,0.0,3,3.005540e+09,0,0053,980567,186739,16a,None,301 003,30206,None,0.0,None,1,1,1,18v2.1,253.999904,2549.997545,"POLYGON ((980614.0479423851 186715.031475395, ..."


In [36]:
len(list(set(reahook_Zoning['BBL'])&(set(redhook['BBL']))))

365

In [28]:
len(list(set(reahook_Zoning['BBL'])))

381

In [29]:
len(list(set(reahook_Zoning['BBL'])))

370

In [33]:
list(set(redhook['BBL']).difference(set(reahook_Zoning['BBL'])))

[3005877501.0, 3006037501.0, 3005757501.0, 3005767501.0, 3005547501.0]

In [34]:
list(set(reahook_Zoning['BBL']).difference(set(redhook['BBL'])))

[3005870050L,
 3005760034L,
 3005870051L,
 3005860004L,
 3005860005L,
 3005850025L,
 3005970027L,
 3005750018L,
 3006030030L,
 3005760016L,
 3005980049L,
 3005870002L,
 3005870003L,
 3005540034L,
 3005760059L,
 3005870013L]